In [34]:
%config IPCompleter.greedy=True
import pandas as pd
import pickle

data = pd.read_csv("./csv/all.csv")
zi = pd.read_pickle('horse_zi.pickle')
data = pd.merge(data, zi, on='レースID(新)', how='inner')
zi_dev = pd.read_pickle('horse_zi_dev.pickle')
data = pd.merge(data, zi_dev, on='レースID(新)', how='inner')
compi = pd.read_pickle('horse_compi.pickle')
data = pd.merge(data, compi, on='レースID(新)', how='inner')
data.to_pickle("ai.pickle")
results = pd.read_pickle("ai.pickle")
results = results[-(results['前走レースID(新)'].astype(str).str.contains("\D"))]
results['前走レースID(新)'] = results['前走レースID(新)'].astype(int)
results.head()

,レースID(新),前走レースID(新),場所,距離,クラス名,出走頭数,着順,騎手,多頭出し,所属,...,4角,上り3F,上り3F順,賞金,付加賞金,スピード指数,レースID(新/馬番無),スピード偏差値,コンピ指数,コンピ順位
0,202205150502081205,202204030902040710,東京,ダ1400,3勝,16,１,川田将雅,NaN,(栗),...,3.0,35.7,7,1840,35.0,118,2022051505020812,65.9,78,1
1,202205150502081212,202204170902081212,東京,ダ1400,3勝,16,２,坂井瑠星,NaN,(栗),...,1.0,36.1,9,740,10.0,113,2022051505020812,60.2,70,3
2,202205150502081207,202204160902071011,東京,ダ1400,3勝,16,３,福永祐一,NaN,(栗),...,9.0,35.3,6,460,5.0,109,2022051505020812,55.7,56,4
3,202205150502081215,202204160902071006,東京,ダ1400,3勝,16,４,武豊,NaN,(栗),...,11.0,35.1,3,280,0.0,108,2022051505020812,54.5,55,5
4,202205150502081206,202203120702011115,東京,ダ1400,3勝,16,５,ルメール,NaN,(美),...,6.0,36.2,10,184,0.0,120,2022051505020812,68.2,75,2


In [35]:
results.columns

Index(['レースID(新)', '前走レースID(新)', '場所', '距離', 'クラス名', '出走頭数', '着順', '騎手',
       '多頭出し', '所属', '調教師', '性別', '年齢', '斤量', '馬名', '種牡馬', '母父馬', '馬体重',
       '馬体重増減', '人気', '単勝オッズ', '枠番', '馬番', '馬連', '３連単', '走破タイム', '着差', '1角',
       '2角', '3角', '4角', '上り3F', '上り3F順', '賞金', '付加賞金', 'スピード指数',
       'レースID(新/馬番無)', 'スピード偏差値', 'コンピ指数', 'コンピ順位'],
      dtype='object')

In [36]:
import re
import datetime

def preprocessing(results):
    df = results.copy()
    df = df[-(df['着順'].astype(str).str.contains("\D"))]
    df = df[-(df['前走レースID(新)'].astype(str).str.contains("\D"))]
    df['前走レースID(新)'] = df['前走レースID(新)'].astype(int)
    df['着順'] = df['着順'].astype(int)
    
    df['斤量'] = df['斤量'].map(lambda x: re.sub('▲|☆|△|◇|★', '', str(x)))
    df['斤量'] = df['斤量'].astype(float)
    
    df['馬体重増減'] = df['馬体重増減'].astype(float)
    
    df['コース'] = df['距離'].map(lambda x: x[:1])
    df['距離'] = df['距離'].map(lambda x: re.split('芝|ダ', x)[1])
    df['date'] = df['レースID(新)'].astype(str).map(lambda x: datetime.datetime.strptime(x[:8], '%Y%m%d'))
    
    df.drop(['出走頭数', '馬体重', 
#              '前走レースID(新)',
                   '多頭出し', '所属', '調教師','種牡馬', '母父馬','馬連', '３連単',
                   '1角', '2角', '3角', '4角','上り3F', '上り3F順', '賞金', '付加賞金'], axis=1, inplace=True)

    return df

def kako_5(race_id, line = None, passed = None, count = 0):
    if line is None:
        line = {'レースID(新)':  race_id}
    
    count = count + 1        
    if count == 10:
        return line
    else:
        base = merge[merge['レースID(新)'] == race_id]

        if passed is None:
            passed = merge[merge['レースID(新)'] == base['前走レースID(新)'].values[0]]
        else:
            passed = merge[merge['レースID(新)'] == passed['前走レースID(新)'].values[0]]

#      前走除外、中止など   
        if len(passed) == 0:
            return line
        
        target_columns = [
            '場所', '距離', 'クラス名', '斤量', '馬体重', '馬体重増減', '着順', '騎手', '騎手コード',
            '人気', '単勝オッズ', '枠番', '馬番', '馬連', '３連単', '走破タイム', '着差',
            '1角','2角', '3角', '4角', '上り3F', '上り3F順', '賞金', '付加賞金', 'コンピ指数', 'コンピ順位', 'スピード指数', 'スピード偏差値'
        ]

        for column in target_columns:
            line[str(count) + "走前" + column] = passed[column].values[0]
#             line[str(count) + '走前着順'] = passed['着順'].values[0]
#             line[str(count) + '走前着差'] = passed['着差'].values[0]
#             line[str(count) + '走前走破タイム'] = passed['走破タイム'].values[0]
#             line[str(count) + '走前人気'] = passed['人気'].values[0]
#             line[str(count) + '走前馬番'] = passed['馬番'].values[0]
#             line[str(count) + '走前騎手'] = passed['騎手'].values[0]

        return kako_5(base['前走レースID(新)'].values[0], line, passed, count)

In [40]:
race_info = pd.read_pickle("ai.pickle")
j = pd.read_csv("./csv/alljockey.csv")
merge = pd.merge(race_info, j, on='レースID(新)', how='inner')
merge.head()

,レースID(新),前走レースID(新),場所,距離,クラス名,出走頭数,着順,騎手,多頭出し,所属,...,上り3F順,賞金,付加賞金,スピード指数,レースID(新/馬番無),スピード偏差値,コンピ指数,コンピ順位,騎手コード,調教師コード
0,202205150502081205,202204030902040710,東京,ダ1400,3勝,16,１,川田将雅,NaN,(栗),...,7,1840,35.0,118,2022051505020812,65.9,78,1,1088,1069
1,202205150502081212,202204170902081212,東京,ダ1400,3勝,16,２,坂井瑠星,NaN,(栗),...,9,740,10.0,113,2022051505020812,60.2,70,3,1163,1151
2,202205150502081207,202204160902071011,東京,ダ1400,3勝,16,３,福永祐一,NaN,(栗),...,6,460,5.0,109,2022051505020812,55.7,56,4,1014,1098
3,202205150502081215,202204160902071006,東京,ダ1400,3勝,16,４,武豊,NaN,(栗),...,3,280,0.0,108,2022051505020812,54.5,55,5,666,1058
4,202205150502081206,202203120702011115,東京,ダ1400,3勝,16,５,ルメール,NaN,(美),...,10,184,0.0,120,2022051505020812,68.2,75,2,5339,1126


In [41]:
merge.to_pickle('all_add_jockey.pickle')

In [42]:
merge = merge[-(merge['前走レースID(新)'].astype(str).str.contains("\D"))]
merge['前走レースID(新)'] = merge['前走レースID(新)'].astype(int)
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 675419 entries, 0 to 687337
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   レースID(新)      675419 non-null  int64  
 1   前走レースID(新)    675419 non-null  int64  
 2   場所            675419 non-null  object 
 3   距離            675419 non-null  object 
 4   クラス名          675419 non-null  object 
 5   出走頭数          675419 non-null  int64  
 6   着順            675419 non-null  object 
 7   騎手            675419 non-null  object 
 8   多頭出し          43119 non-null   object 
 9   所属            675419 non-null  object 
 10  調教師           675419 non-null  object 
 11  性別            675419 non-null  object 
 12  年齢            675419 non-null  int64  
 13  斤量            675419 non-null  object 
 14  馬名            675419 non-null  object 
 15  種牡馬           675419 non-null  object 
 16  母父馬           675419 non-null  object 
 17  馬体重           674609 non-null  float64
 18  馬体重増

In [43]:
race_ids = results['レースID(新)'].astype(int)

In [44]:
a = []
for race_id in race_ids:
    a.append(kako_5(race_id))
pd.DataFrame(a).head()

,レースID(新),1走前場所,1走前距離,1走前クラス名,1走前斤量,1走前馬体重,1走前馬体重増減,1走前着順,1走前騎手,1走前騎手コード,...,9走前3角,9走前4角,9走前上り3F,9走前上り3F順,9走前賞金,9走前付加賞金,9走前コンピ指数,9走前コンピ順位,9走前スピード指数,9走前スピード偏差値
0,202205150502081205,阪神,ダ1400,2勝,57,512.0,2.0,１,川田将雅,1088.0,...,4.0,6.0,38.6,3.0,130.0,NaN,58.0,4.0,109.0,58.9
1,202205150502081212,阪神,ダ1400,2勝,57,538.0,-4.0,１,坂井瑠星,1163.0,...,1.0,1.0,39.5,10.0,510.0,NaN,59.0,5.0,111.0,56.6
2,202205150502081207,阪神,ダ1400,3勝,57,502.0,0.0,７,福永祐一,1014.0,...,2.0,1.0,39.8,8.0,760.0,NaN,90.0,1.0,130.0,62.1
3,202205150502081215,阪神,ダ1400,3勝,57,444.0,-6.0,５,武豊,666.0,...,11.0,9.0,37.3,7.0,0.0,0.0,61.0,3.0,100.0,51.5
4,202205150502081206,中京,ダ1400,3勝,54,452.0,-8.0,２,泉谷楓真,1182.0,...,5.0,6.0,34.0,3.0,77.0,NaN,71.0,2.0,115.0,62.8


In [45]:
pd.DataFrame(a).to_pickle('kako_10.pickle')

In [46]:
df_from_pkl = pd.read_pickle('kako_10.pickle')
# merge1 = pd.merge(merge, df_from_pkl, on='レースID(新)', how='inner')

In [47]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
df_from_pkl.head()

,レースID(新),1走前場所,1走前距離,1走前クラス名,1走前斤量,1走前馬体重,1走前馬体重増減,1走前着順,1走前騎手,1走前騎手コード,...,9走前3角,9走前4角,9走前上り3F,9走前上り3F順,9走前賞金,9走前付加賞金,9走前コンピ指数,9走前コンピ順位,9走前スピード指数,9走前スピード偏差値
0,202205150502081205,阪神,ダ1400,2勝,57,512.0,2.0,１,川田将雅,1088.0,...,4.0,6.0,38.6,3.0,130.0,NaN,58.0,4.0,109.0,58.9
1,202205150502081212,阪神,ダ1400,2勝,57,538.0,-4.0,１,坂井瑠星,1163.0,...,1.0,1.0,39.5,10.0,510.0,NaN,59.0,5.0,111.0,56.6
2,202205150502081207,阪神,ダ1400,3勝,57,502.0,0.0,７,福永祐一,1014.0,...,2.0,1.0,39.8,8.0,760.0,NaN,90.0,1.0,130.0,62.1
3,202205150502081215,阪神,ダ1400,3勝,57,444.0,-6.0,５,武豊,666.0,...,11.0,9.0,37.3,7.0,0.0,0.0,61.0,3.0,100.0,51.5
4,202205150502081206,中京,ダ1400,3勝,54,452.0,-8.0,２,泉谷楓真,1182.0,...,5.0,6.0,34.0,3.0,77.0,NaN,71.0,2.0,115.0,62.8


In [17]:
# results = pd.read_pickle("ai.pickle")
# kako = pd.read_pickle('kako_10.pickle')
# merge = pd.merge(results, kako, on='レースID(新)', how='inner')
# merge.head()

%config IPCompleter.greedy=True
import pandas as pd
import pickle
# result_all = pd.read_pickle('allrace_peds_raceid2.pickle')
# result_all = result_all.reset_index()
# result_all.drop(['horse_id'], axis=1, inplace=True)
# horse_ids = pd.read_pickle("horseid.pickle")
# result_all = pd.merge(result_all, horse_ids, on='レースID(新)', how='inner')

# peds = pd.read_pickle("all_peds.pickle")
# result_all['血統登録番号'] = result_all['血統登録番号'].astype(str)
# results2 = result_all.merge(peds, left_on='血統登録番号', right_index=True, how='left')
# results2['horse_id'] = results2['血統登録番号']
# results2['horse_id'] = results2['horse_id'].astype(int)
# results2.drop(['血統登録番号'], axis=1, inplace=True)
# raceid = pd.read_csv("./csv/raceid.csv")
# hoge = pd.merge(results2, raceid, on='レースID(新)', how='inner')
# hoge = hoge.drop_duplicates(['前走レースID(新)']).reset_index()

# hoge.to_pickle('allrace_peds_raceid2.pickle')
hoge.to_pickle('allrace_peds_raceid2.pickle')

In [194]:
for num in range(1, 10):
    df = df_from_pkl.copy()
    df[str(num) + '走前着差'] = pd.to_numeric(df[str(num) + '走前着差'], errors='ignore')
#     df = df[df[str(num) + '走前着差'].astype(str).str.contains("[+-]?\d+(?:\.\d+)?")]
#     df[str(num)+'走前着差'] = df[str(num)+'走前着差'].astype(float)
    df[str(num) + '走前斤量'] = df[str(num) + '走前斤量'].map(lambda x: re.sub('▲|☆|△|◇|★', '', str(x)))
    df[str(num) + '走前斤量'] = df[str(num) + '走前斤量'].astype(float)

<class 'pandas.core.series.Series'>
RangeIndex: 664685 entries, 0 to 664684
Series name: 1走前着差
Non-Null Count   Dtype 
--------------   ----- 
575451 non-null  object
dtypes: object(1)
memory usage: 5.1+ MB


In [4]:
import pandas as pd
import pickle
raceid = pd.read_csv("./csv/raceid.csv")
haitou = pd.read_csv("./csv/haitou.csv")
race_haitou = pd.merge(raceid, haitou, on='レースID(新/馬番無)', how='inner')
race_haitou = race_haitou[-race_haitou['３連単'].isna()]
race_haitou['３連単'] = race_haitou['３連単'].astype(int)
race_haitou = race_haitou.drop_duplicates(subset=['レースID(新/馬番無)']).reset_index(drop=True)
race_haitou.drop(['レースID(新)'], axis=1, inplace=True)
# race_haitou.to_pickle('haitou.pickle')
race_haitou[]

,レースID(新/馬番無),単勝配当,馬連,馬単,３連複,３連単
0,2022051505020812,410,1410,2700,3870,17210
1,2022051505020811,570,2010,4010,8270,43780
2,2022051505020810,990,3660,6930,25800,140230
3,2022051505020809,1270,3550,8430,5710,50350
4,2022051505020808,680,1390,2740,7330,31050
...,...,...,...,...,...,...
47815,2007010606010109,730,8630,14980,73050,439550
47816,2007010608010112,380,720,1410,5900,23850
47817,2007010608010111,1220,7020,16910,13720,119220
47818,2007010608010110,350,890,1690,2660,10350


In [303]:
race_haitou = pd.read_pickle('haitou.pickle')
raceid = pd.read_csv("./csv/raceid.csv")
race_haitou = pd.merge(raceid, haitou, on='レースID(新/馬番無)', how='inner')
race_haitou.head()

,レースID(新),レースID(新/馬番無),単勝配当,馬連,馬単,３連複,３連単
0,202205150502081205,2022051505020812,410,1410,2700,3870,17210.0
1,202205150502081212,2022051505020812,410,1410,2700,3870,17210.0
2,202205150502081207,2022051505020812,410,1410,2700,3870,17210.0
3,202205150502081215,2022051505020812,410,1410,2700,3870,17210.0
4,202205150502081206,2022051505020812,410,1410,2700,3870,17210.0


In [332]:
raceid

,レースID(新),レースID(新/馬番無)
0,202205150502081205,2022051505020812
1,202205150502081212,2022051505020812
2,202205150502081207,2022051505020812
3,202205150502081215,2022051505020812
4,202205150502081206,2022051505020812
...,...,...
759520,200701060801010104,2007010608010101
759521,200701060801010115,2007010608010101
759522,200701060801010103,2007010608010101
759523,200701060801010114,2007010608010101


In [16]:
fukusho = pd.read_csv('./csv/fukusho.csv')
fukusho = fukusho[-fukusho['複勝配当'].isna()]
fukusho['複勝配当'] = fukusho['複勝配当'].astype(int)

In [17]:
fukushos = []
for race_id in fukusho['レースID(新/馬番無)'].unique():
    generations = {'レースID(新/馬番無)': race_id}
    count = 1
    
    for item in fukusho[fukusho['レースID(新/馬番無)'] == race_id]['複勝配当']:
        generations['複勝_' + str(count)] = item
        count = count + 1
        
    count = 1
    for num in fukusho[fukusho['レースID(新/馬番無)'] == race_id]['馬番']:
        generations['馬番_' + str(count)] = num
        count = count + 1
        
    fukushos.append(generations)
fukushos = pd.DataFrame(fukushos)

In [18]:
fukushos = fukushos.fillna(0)

In [19]:
fukushos['複勝_3'] = fukushos['複勝_3'].astype(int)
fukushos['複勝_4'] = fukushos['複勝_4'].astype(int)
fukushos['馬番_3'] = fukushos['馬番_3'].astype(int)
fukushos['馬番_4'] = fukushos['馬番_4'].astype(int)
fukushos

,レースID(新/馬番無),複勝_1,複勝_2,複勝_3,馬番_1,馬番_2,馬番_3,複勝_4,馬番_4
0,2022051505020812,180,230,260,5,12,7,0,0
1,2022051505020811,230,190,360,5,11,7,0,0
2,2022051505020810,340,220,580,3,10,12,0,0
3,2022051505020809,340,190,180,10,5,7,0,0
4,2022051505020808,190,180,420,1,6,3,0,0
...,...,...,...,...,...,...,...,...,...
53119,2007010608010105,120,100,140,14,9,8,0,0
53120,2007010608010104,150,220,240,5,8,11,0,0
53121,2007010608010103,170,150,140,2,5,1,0,0
53122,2007010608010102,220,220,760,1,8,11,0,0


In [22]:
hoge = race_haitou.drop_duplicates(subset=['レースID(新/馬番無)']).reset_index(drop=True)
haito = pd.merge(hoge, fukushos, on='レースID(新/馬番無)', how='inner')
# haito = haito.drop(['レースID(新)'], axis=1)
haito = haito.set_index('レースID(新/馬番無)')

In [25]:
haito.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47819 entries, 2022051505020812 to 2007010608010109
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   単勝配当    47819 non-null  int64
 1   馬連      47819 non-null  int64
 2   馬単      47819 non-null  int64
 3   ３連複     47819 non-null  int64
 4   ３連単     47819 non-null  int64
 5   複勝_1    47819 non-null  int64
 6   複勝_2    47819 non-null  int64
 7   複勝_3    47819 non-null  int64
 8   馬番_1    47819 non-null  int64
 9   馬番_2    47819 non-null  int64
 10  馬番_3    47819 non-null  int64
 11  複勝_4    47819 non-null  int64
 12  馬番_4    47819 non-null  int64
dtypes: int64(13)
memory usage: 5.1 MB


In [24]:
haito.to_pickle('haito.pickle')

In [299]:
fukushos['レースID(新/馬番無)'] = fukushos['複勝_レースID(新/馬番無)']
fukushos = fukushos.drop(['複勝_レースID(新/馬番無)'], axis=1)

In [334]:
race_haitou = pd.merge(race_haitou, fukushos, on='レースID(新/馬番無)', how='inner')
haito

,レースID(新/馬番無),単勝配当,馬連,馬単,３連複,３連単,複勝_1,複勝_2,複勝_3,馬番_1,馬番_2,馬番_3,複勝_4,馬番_4
0,2022051505020812,410,1410,2700,3870,17210.0,180,230,260,5,12,7,0,0
1,2022051505020811,570,2010,4010,8270,43780.0,230,190,360,5,11,7,0,0
2,2022051505020810,990,3660,6930,25800,140230.0,340,220,580,3,10,12,0,0
3,2022051505020809,1270,3550,8430,5710,50350.0,340,190,180,10,5,7,0,0
4,2022051505020808,680,1390,2740,7330,31050.0,190,180,420,1,6,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51141,2007010608010105,350,340,870,570,NaN,120,100,140,14,9,8,0,0
51142,2007010608010104,390,1100,1960,2810,NaN,150,220,240,5,8,11,0,0
51143,2007010608010103,600,1050,2610,1220,NaN,170,150,140,2,5,1,0,0
51144,2007010608010102,570,1470,3350,8150,NaN,220,220,760,1,8,11,0,0


In [302]:
race_haitou.to_pickle('race_haitou.pickle')

In [574]:
clip_rank = lambda x: x if x < 4 else 4

test['rank'] = test['着順'].map(clip_rank)

In [575]:
test.drop(['着順', '馬名'], axis=1, inplace=True)
test_d = pd.get_dummies(test).dropna(how='any')

In [576]:
test_d

,レースID(新),年齢,斤量,馬体重増減,人気,単勝オッズ,枠番,馬番,date,rank,...,着差_9.2,着差_9.3,着差_9.4,着差_9.5,着差_9.6,着差_9.7,着差_9.8,着差_9.9,コース_ダ,コース_芝
0,202205150502081205,4,55.0,-2.0,2.0,4.1,3,5,2022-05-15,1,...,0,0,0,0,0,0,0,0,1,0
1,202205150502081212,4,55.0,-12.0,3.0,6.4,6,12,2022-05-15,2,...,0,0,0,0,0,0,0,0,1,0
2,202205150502081207,5,56.0,-2.0,4.0,7.5,4,7,2022-05-15,3,...,0,0,0,0,0,0,0,0,1,0
3,202205150502081215,6,56.0,-2.0,7.0,25.9,8,15,2022-05-15,4,...,0,0,0,0,0,0,0,0,1,0
4,202205150502081206,4,54.0,8.0,1.0,3.0,3,6,2022-05-15,4,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734493,200701060801010104,3,54.0,4.0,9.0,34.4,2,4,2007-01-06,4,...,0,0,0,0,0,0,0,0,1,0
734494,200701060801010115,3,56.0,-6.0,15.0,281.9,8,15,2007-01-06,4,...,0,0,0,0,0,0,0,0,1,0
734495,200701060801010103,3,56.0,22.0,14.0,191.6,2,3,2007-01-06,4,...,0,0,0,0,0,0,0,0,1,0
734496,200701060801010114,3,54.0,0.0,11.0,44.6,7,14,2007-01-06,4,...,0,0,0,0,0,0,0,0,1,0


In [16]:
test

,レースID(新/馬番無),場所,距離,着順,騎手,性別,年齢,斤量,馬名,馬体重増減,人気,単勝オッズ,枠番,馬番,コース,date
0,2022051505020812,東京,1400,1,川田将雅,牡,4,55.0,エイシンギアアップ,-2.0,2.0,4.1,3,5,1,2022/05/15
1,2022051505020812,東京,1400,2,坂井瑠星,セ,4,55.0,ローウェル,-12.0,3.0,6.4,6,12,1,2022/05/15
2,2022051505020812,東京,1400,3,福永祐一,牡,5,56.0,ファイアランス,-2.0,4.0,7.5,4,7,1,2022/05/15
3,2022051505020812,東京,1400,4,武豊,牡,6,56.0,シェパードボーイ,-2.0,7.0,25.9,8,15,1,2022/05/15
4,2022051505020812,東京,1400,5,ルメール,牝,4,54.0,グランパラディーゾ,8.0,1.0,3.0,3,6,1,2022/05/15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510309,2012010508010101,京都,1200,12,国分優作,牡,3,55.0,クールフォックス,0.0,10.0,117.8,8,16,1,2012/01/05
510310,2012010508010101,京都,1200,13,岩崎祐己,牡,3,56.0,メイショウウイナー,12.0,13.0,275.5,6,12,1,2012/01/05
510311,2012010508010101,京都,1200,14,水口優也,牡,3,53.0,アストゥリアス,14.0,15.0,508.2,7,13,1,2012/01/05
510312,2012010508010101,京都,1200,15,福永祐一,牡,3,56.0,アシュケナージ,12.0,3.0,7.9,6,11,1,2012/01/05


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = test_d.drop(['rank'], axis=1)
y = test_d['rank']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [20]:
X_train

,年齢,斤量,馬体重増減,人気,単勝オッズ,枠番,馬番,騎手_アッゼニ,騎手_アヴドゥ,騎手_ウィリア,...,騎手_鴨宮祥行,騎手_鷲頭虎太,騎手_黒岩悠,騎手_黒沢愛斗,騎手_黛弘人,騎手_Ｃ．デム,騎手_Ｍ．デム,性別_セ,性別_牝,性別_牡
457586,5,57.0,4.0,10.0,63.3,8,16,0,0,0,...,0,0,0,0,0,0,0,0,0,1
219191,3,54.0,2.0,1.0,3.5,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
394510,3,56.0,0.0,1.0,2.3,8,9,0,0,0,...,0,0,0,0,0,0,0,0,0,1
100248,3,52.0,0.0,8.0,25.1,3,5,0,0,0,...,0,0,0,0,0,0,0,0,1,0
39871,2,54.0,2.0,3.0,7.6,6,6,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389209,6,58.0,0.0,7.0,36.4,2,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
450365,3,56.0,-2.0,2.0,6.3,8,15,0,0,0,...,0,0,0,0,0,0,0,0,0,1
145297,3,56.0,0.0,10.0,153.6,5,9,0,0,0,...,0,0,0,0,0,0,0,0,0,1
274122,5,55.0,0.0,15.0,228.0,6,11,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
y_pred = model.predict(X_test)

In [24]:
pd.Series(y_pred).value_counts()

4    137729
dtype: int64

In [25]:
from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1: rank_1, 2: rank_2, 3: rank_3, 4: rank_1}, random_state=7)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [279]:
pd.Series(y_train_rus).value_counts()

2    22940
3    22937
1    22924
4    22924
Name: rank, dtype: int64

In [26]:
model = LogisticRegression()
model.fit(X_train_rus, y_train_rus)

print(model.score(X_train, y_train), model.score(X_test, y_test))

/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5516154427803726 0.5491000442898736


In [27]:
y_pred = model.predict(X_test)

In [28]:
pred_df = pd.DataFrame({"pred": y_pred, "actual": y_test})

In [30]:
pred_df[pred_df['pred'] == 1]["actual"].value_counts()

4    15182
1     6495
2     5313
3     4162
Name: actual, dtype: int64

In [31]:
len(pred_df[pred_df['pred'] == 1])

31152

In [32]:
pred_df[pred_df['pred'] == 4]["actual"].value_counts()

4    65806
3     2171
2     1576
1      965
Name: actual, dtype: int64

In [287]:
len(pred_df[pred_df['pred'] == 4])

70518

In [33]:
coef = pd.Series(model.coef_[0], index=X.columns).sort_values()

In [34]:
coef

人気        -0.159125
枠番        -0.022823
年齢        -0.010604
騎手_和田竜二   -0.006204
騎手_岩田康誠   -0.005329
             ...   
騎手_戸崎圭太    0.005469
騎手_川田将雅    0.008354
騎手_ルメール    0.013510
馬番         0.014861
斤量         0.015335
Length: 423, dtype: float64

In [290]:
test[test['騎手'] == 'ルメール']['rank'].value_counts()

4    2789
1    1397
2     967
3     690
Name: rank, dtype: int64

In [239]:
test.columns

Index(['騎手', '性別', '年齢', '斤量', '馬体重増減', '人気', '単勝オッズ', '枠番', '馬番', 'rank'], dtype='object')

In [291]:
coef[[ '年齢', '斤量', '馬体重増減', '人気', '単勝オッズ', '枠番', '馬番']]

年齢      -0.010604
斤量       0.015335
馬体重増減    0.001513
人気      -0.159125
単勝オッズ   -0.001591
枠番      -0.022823
馬番       0.014861
dtype: float64